In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [16]:

# Load the data
data = pd.read_csv('jobs_in_data.csv')

# Drop the 'salary_in_usd' column since we're ignoring it
data.drop(columns=['salary_in_usd'], inplace=True)

# Encode categorical variables
data = pd.get_dummies(data)
# Divide the 'salary' column by 1000
data['salary'] /= 1000
# Separate features and target
X = data.drop(columns=['salary'])
y = data['salary']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    # Dropout(0.2),
    Dense(64, activation='relu'),
    # Dropout(0.2),
    Dense(1, activation='linear')
])

# Compile the model with a custom learning rate
# custom_learning_rate = 0.001  # Set your desired learning rate here
# model.compile(optimizer=Adam(learning_rate=custom_learning_rate), loss='mean_squared_error')
model.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the model
model.fit(X_train_scaled, y_train, epochs=1000, batch_size=32, validation_split=0.1)

# Evaluate the model
loss = model.evaluate(X_test_scaled, y_test)
print('Test Loss:', loss)

# Make predictions on the test set
predictions = model.predict(X_test_scaled).flatten()

# Visualize Predictions vs. Actual Values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, predictions, alpha=0.5)
plt.xlabel('Actual Salary')
plt.ylabel('Predicted Salary')
plt.title('Actual vs. Predicted Salaries')
plt.show()

# Calculate Evaluation Metrics
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print('Mean Absolute Error:', mae)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)

# Feature Importance
# Get the weights of the first layer
weights = model.layers[0].get_weights()[0]

# Calculate the importance of features based on the weights
feature_importance = np.mean(np.abs(weights), axis=0)

# Get feature names
feature_names = X.columns

# Sort features by importance
sorted_indices = np.argsort(feature_importance)[::-1]

# Top 10 features
top_features = feature_names[sorted_indices][:10]
print('Top 10 Features:', top_features)

Epoch 1/1000
211/211 [==============================] - 3s 7ms/step - loss: 9494.7832 - val_loss: 2688.7422
Epoch 2/1000
211/211 [==============================] - 1s 7ms/step - loss: 2549.1687 - val_loss: 2656.9692
Epoch 3/1000
211/211 [==============================] - 2s 9ms/step - loss: 2416.0188 - val_loss: 2561.8525
Epoch 4/1000
211/211 [==============================] - 1s 7ms/step - loss: 2363.3018 - val_loss: 2555.0652
Epoch 5/1000
211/211 [==============================] - 1s 7ms/step - loss: 2334.1072 - val_loss: 2522.6997
Epoch 6/1000
211/211 [==============================] - 1s 7ms/step - loss: 2313.1965 - val_loss: 2510.1963
Epoch 7/1000
211/211 [==============================] - 1s 6ms/step - loss: 2296.4270 - val_loss: 2479.1816
Epoch 8/1000
211/211 [==============================] - 1s 6ms/step - loss: 2284.6484 - val_loss: 2494.1389
Epoch 9/1000
211/211 [==============================] - 1s 6ms/step - loss: 2287.3379 - val_loss: 2456.5986
Epoch 10/1000
211/211 [=====

KeyboardInterrupt: 

 2813702912.0 0.001
 2603507200.0 0.01
 3033694464.0 0.1
 2603507017.888839 0.01

 1000 epoch
 2596519395.2590485
root 50956.05356833522
